# **Νευρωνικά Δίκτυα και Ευφυή Υπολογιστικά Συστήματα**

## **Άσκηση 1. Επιβλεπόμενη Μάθηση: Ταξινόμηση. Μελέτη datasets του UCI Machine Learning Repository**
#### Μαρίνος Τζανίκος 03113147, el13147@mail.ntua.gr
#### Γιώργος Γκανάς 03116095, el16095@mail.ntua.gr
#### Παναγιώτης Αϊβασιλιώτης 03116176, el16176@mail.ntua.gr

In [ ]:
!pip install --upgrade pip 
!pip install --upgrade scikit-learn 
!pip install --upgrade numpy 
!pip install --upgrade scipy 
!pip install --upgrade pandas 
!pip install --upgrade imbalanced-learn

import warnings 
warnings.filterwarnings('ignore')

###**Μικρό Dataset(S)**

####**Βασικές πληροφορίες**

**Εισαγωγή του dataset**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("crx.data", header=None)
#Show the first 5 samples of the dataframe
df.head(n=5)

**Παρουσίαση του dataset**

To dataset περιγράφει την έκβαση των αιτήσεων για πιστωτικές κάρτες με βάση κάποια χαρακτηριστικά.Δηλαδή αν η αίτηση κάποιου για πιστωτική κάρτα, εγκρίθηκε ή όχι.Στόχος μας είναι με χρήση επιβλεπόμενης μάθησης και χωρίς να ξέρουμε την πραγματική ερμηνεία των δεδομένων να δημιουργήσουμε ένα συστήμα που θα μπορεί να χαρακτηρίζει όσο το δυνατόν καλύτερα αν η έκβαση μίας υποψηφίας αίτησης θα είναι θετική ή αρνητική.

**Αριθμός δειγμάτων και χαρακτηριστικών, είδος χαρακτηριστικών. Υπάρχουν μη διατεταγμένα χαρακτηριστικά και ποια είναι αυτά;**

Το dataset περιέχει 690 δείγματα.Kάθε δείγμα αποτελείται από 15 χαρακτηριστικά τα οποία είναι κατηγορικά ή αριθμητικά και των οποίων η πραγματική ερμηνεία δεν μας δίνεται.Τα διατεταγμένα χαρακτηριστικά είναι μόνο τα αριθμητικά.Ακόμα κάθε δείγμα περιέχει μία ετικέτα που μας δείχνει την έκβαση που είχε η εν λόγω αίτηση.Πιο συγκεκριμένα έχουμε τα εξής σχετικά με τις πληροφορίες που περιέχει ένα δείγμα:

A1:	b, a.  
A2:	continuous.  
A3:	continuous.  
A4:	u, y, l, t.  
A5:	g, p, gg.  
A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.  
A7:	v, h, bb, j, n, z, dd, ff, o.  
A8:	continuous.  
A9: t, f.  
A10: t, f.  
A11: continuous.  
A12: t, f.  
A13: g, p, s.  
A14: continuous.  
A15: continuous.  
A16: +,- (class attribute)

Επομένως έχουμε οτι τα χαρακτηριστικά Α2,Α3,Α8,Α11,Α14,Α15 είναι συνεχή και τα υπόλοιπα 10 κατηγορικά.

**Υπάρχουν επικεφαλίδες; Αρίθμηση γραμμών;**

Όχι, τα δεδομένα μας βρίσκονται σε raw μορφή μέσα στο αρχείο και επομένως δεν έχουν ούτε επικεφαλίδες ούτε αρίθμηση γραμμών.

**Ποιες είναι οι ετικέτες των κλάσεων και σε ποια κολόνα βρίσκονται;**

Οι ετικέτες των κλάσεων βρίσκονται στην 16η κολόνα και είναι είτε "+" δηλαδή θετική έκβαση της αίτησης είτε "-" δηλαδή αρνητική έκβαση της αίτησης.

In [ ]:
unique_labels = df.iloc[:,-1].unique()
print("Unique labels of samples: {0}".format(unique_labels))

**Χρειάστηκε να κάνετε μετατροπές στα αρχεία text και ποιες?**

Οχι δεν χρειάστηκε να γίνει κάποια μετατροπή στα αρχεία text.Ωστόσο κρίθηκε απαραίτητο να γίνει μία κωδικοποίηση των ετικετών κλάσης από -,+ σε 0,1 ώστε τα δεδομένα μας να είναι συμβατά με το scikit-learn.

In [ ]:
labels_df = df.iloc[:,-1] # labels in the last column
features_df = df.iloc[:,:-1] # features in all columns but the last

mapping = {'-' : 0, '+' : 1} # mapping for labels
labels_df = labels_df.replace(mapping)

**Υπάρχουν απουσιάζουσες τιμές; Πόσα είναι τα δείγματα με απουσιάζουσες τιμές και ποιο το ποσοστό τους επί του συνόλου;**

Παρατηρούμε ότι οι απουσιάζουσες τιμές εμφανίζονται με "?" μέσα στο αρχείο εισόδου και δεν υπάρχει καμία απουσιάζουσα ετικέτα.

In [ ]:
df.replace('?', np.NaN, inplace=True) # replace all ? with np.NaN


In [ ]:
df.isnull().sum().sum() # how many missing values we have

In [ ]:
sum([True for idx,row in df.iterrows() if any(row.isnull())]) # how many samples have missing values

Επομένως υπάρχουν συνολικά 67 απουσιάζουσες τιμές σε 37 εκ των δειγμάτων, που αντιστοιχούν στο 5,3% του συνόλου.

**Ποιος είναι ο αριθμός των κλάσεων και τα ποσοστά δειγμάτων τους επί του συνόλου; Αν θεωρήσουμε ότι ένα dataset είναι μη ισορροπημένο αν μια οποιαδήποτε κλάση είναι 1.5 φορά πιο συχνή από κάποια άλλη (60%-40% σε binary datasets) εκτιμήστε την ισορροπία του dataset.**

In [ ]:
np_labels = labels_df.values.flatten() # convert labels dataframe to numpy array
frequencies = np.bincount(np_labels)
total_samples = np_labels.shape[0]
percentages = (frequencies / total_samples) * 100

print("Class frequencies: {0}".format(frequencies))

print("Class percentages: {0}".format(percentages))

Βλέπουμε ότι τα ποσοστά δειγμάτων επί του συνόλου των δύο κλάσεων βρίσκονται κοντά στο 50%, επομένως το dataset είναι ισορροπημένο.

**Διαχωρίστε σε train και test set. Εάν υπάρχουν απουσιάζουσες τιμές και μη διατεταγμένα
χαρακτηριστικά διαχειριστείτε τα και αιτιολογήστε τις επιλογές σας.**

Αφού χωρίσουμε το dataset σε train και test set θα αντικαταστήσουμε τις απουσιάζουσες τιμές καθώς η είσοδος στους αλγορίθμους μηχανικής μάθησης πρέπει να είναι πλήρης.Για το λόγο αυτόν,
Θα χρησιμοποιήσουμε το μετασχηματιστή “Imputer” του scikit learn που αντικαθιστά κάθε απουσιάζουσα τιμή χαρακτηριστικού με τη μέση τιμή (συνεχείς μεταβλητές) ή την πιο συχνή τιμή (κατηγορικές μεταβλητές).
Ο μετασχηματισμός με Imputer γίνεται στην απόλυτη αρχή της προεπεξεργασίας.
Ακόμα πρέπει να

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

#Split our dataframe intο train and test set
trainData, testData, trainLabels, testLabels = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.2,stratify=df.iloc[:,-1],random_state=17)

#Split dataframes into categorical and continuous features dataframes
trainData_continuous = trainData.drop([0,3,4,5,6,8,9,11,12], axis=1)
trainData_categorical = trainData.drop([1,2,7,10,13,14], axis=1)
testData_continuous = testData.drop([0,3,4,5,6,8,9,11,12], axis=1)
testData_categorical = testData.drop([1,2,7,10,13,14], axis=1)

#Impute the missing values on continuous features
imp = SimpleImputer(missing_values=np.NaN,strategy='mean')
itrainData_continuous = pd.DataFrame(imp.fit_transform(trainData_continuous))
itrainData_continuous.columns=[1,2,7,10,13,14]
itestData_continuous = pd.DataFrame(imp.fit_transform(testData_continuous))
itestData_continuous.columns=[1,2,7,10,13,14]

#Impute the missing values on categorical features
imp = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
itrainData_categorical = pd.DataFrame(imp.fit_transform(trainData_categorical))
itrainData_categorical.columns=[0,3,4,5,6,8,9,11,12]
itestData_categorical = pd.DataFrame(imp.fit_transform(testData_categorical))
itestData_categorical.columns=[0,3,4,5,6,8,9,11,12]

#Merge dataframes and reorder indexes to form the original ones imputed
itrainData = pd.concat([itrainData_categorical, itrainData_continuous], axis=1)
itrainData = itrainData.reindex(sorted(itrainData.columns), axis=1)
itestData = pd.concat([itestData_categorical, itestData_continuous], axis=1)
itestData = itestData.reindex(sorted(itestData.columns), axis=1)

#Convert categorical features to binary , we have to join the two data before this step
Data_dummies = pd.get_dummies(pd.concat([itrainData,itestData], join="outer", axis=0),columns=[0,3,4,5,6,8,9,11,12])
trainData_dummies = Data_dummies.iloc[:itrainData.shape[0],:]
testData_dummies = Data_dummies.iloc[itrainData.shape[0]:,:]

#Convert to numeric values and numpy arrays
np_trainData = trainData_dummies.apply(pd.to_numeric).values
np_testData = testData_dummies.apply(pd.to_numeric).values
np_trainLabels = trainLabels.values.flatten()
np_testLabels =  testLabels.values.flatten()

#Giving symbolic ml names to our arrays 
X_train = np_trainData
y_train = np_trainLabels
X_test =  np_testData
y_test =  np_testLabels

#Four lists to save some metrics along the way
before_micro = []
before_macro = []
after_micro = []
after_macro = []

####**Baseline classification**

Αρχικά υλοποιήσαμε τους ταξινομητές που ζητούνται χωρίς καμία βελτιστοποίηση στο κόμματι του preproccesing ή των υπερπαραμέτρων του ταξινομητή.

Για κάθε ταξινομητή που ζητείται υπολογίσαμε τις μετρικές f1_micro avg και f1_macro avg καθώς και τους πίνακες σύγκρισης για την εκτίμηση του στο test set.Ακόμα για τους ταξινομητές kNN και GNB
υπολογίζουμε και την μετρική f1_macro avg με χρηση 10-fold cross validation στο training set ώστε να συγκρίνουμε την απόδοση της βελτιστοποίησης που θα γίνει σε επόμενο βήμα.

Για την εμφάνιση των μετρικών χρησιμοποιήσαμε την συνάρτηση `classification_report` του sklearn η οποία εμφανίζει τις μετρικές precision, recall, f1, support για micro avg, macro avg, weighted avg  συνολικά καθώς και για κάθε κλάση ξεχωριστά, υπογραμμίζουμε οτι το micro avg εμφανίζεται ως accuracy.

Τέλος παρατηρήσαμε πως το sklearn χρησιμοποιεί StratifiedKFold ως default επιλογή στα binary classification.

#####**Dummy Classifiers**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.dummy import DummyClassifier

dc_uniform = DummyClassifier(strategy="uniform")
dc_constant_minus = DummyClassifier(strategy="constant", constant="-")
dc_constant_plus = DummyClassifier(strategy="constant", constant="+")
dc_most_frequent = DummyClassifier(strategy="most_frequent")
dc_stratified = DummyClassifier(strategy="stratified")

dc_uniform.fit(X_train, y_train)
dc_constant_minus.fit(X_train, y_train)
dc_constant_plus.fit(X_train, y_train)
dc_most_frequent.fit(X_train, y_train)
dc_stratified.fit(X_train, y_train)

pred_uni = dc_uniform.predict(X_test)
pred_const_minus = dc_constant_minus.predict(X_test)
pred_const_plus = dc_constant_plus.predict(X_test)
pred_freq = dc_most_frequent.predict(X_test)
pred_strat = dc_stratified.predict(X_test)

before_micro.append(precision_recall_fscore_support(y_test, pred_uni, average='micro')[2])
before_micro.append(precision_recall_fscore_support(y_test, pred_const_minus, average='micro')[2])
before_micro.append(precision_recall_fscore_support(y_test, pred_const_plus, average='micro')[2])
before_micro.append(precision_recall_fscore_support(y_test, pred_freq, average='micro')[2])
before_micro.append(precision_recall_fscore_support(y_test, pred_strat, average='micro')[2])

before_macro.append(precision_recall_fscore_support(y_test, pred_uni, average='macro', zero_division=0)[2])
before_macro.append(precision_recall_fscore_support(y_test, pred_const_minus, average='macro', zero_division=0)[2])
before_macro.append(precision_recall_fscore_support(y_test, pred_const_plus, average='macro', zero_division=0)[2])
before_macro.append(precision_recall_fscore_support(y_test, pred_freq, average='macro', zero_division=0)[2])
before_macro.append(precision_recall_fscore_support(y_test, pred_strat, average='macro', zero_division=0)[2])

print("The DummyClassifier with uniform strategy has score on test set : \n" + classification_report(y_test,pred_uni))
print("The DummyClassifier with constant minus strategy has score on test set : \n" + classification_report(y_test,pred_const_minus))
print("The DummyClassifier with constant plus strategy  has score on test set : \n" + classification_report(y_test,pred_const_plus))
print("The DummyClassifier with frequent strategy has score on test set : \n" + classification_report(y_test,pred_freq))
print("The DummyClassifier with stratified strategy  has score on test set : \n" + classification_report(y_test,pred_strat))

#####**Gausian Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
score_gnb = cross_val_score(gnb, X_train, y_train, cv=10, scoring='f1_macro')
pred_gnb = gnb.predict(X_test)

before_micro.insert(0,precision_recall_fscore_support(y_test, pred_gnb, average='micro')[2])
before_macro.insert(0,precision_recall_fscore_support(y_test, pred_gnb, average='macro', zero_division=0)[2])

print("The GaussianNB Classifier has f1_macro score on training set : %f \n" % (score_gnb.mean()))
print("The GaussianNB Classifier has score on test set : \n" + classification_report(y_test,pred_gnb))

#####**k Nearest Neighbors Classifier (kNN) με k=5**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# kNN Classifier, default k=5
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
score_knn = cross_val_score(knn, X_train, y_train, cv=10, scoring="f1_macro")
pred_knn = knn.predict(X_test)

before_micro.insert(0,precision_recall_fscore_support(y_test, pred_knn, average='micro')[2])
before_macro.insert(0,precision_recall_fscore_support(y_test, pred_knn, average='macro', zero_division=0)[2])

print("The KNeighborsClassifier with k=" + str(knn.n_neighbors) + " has f1_macro score on training set : %f \n" % (score_knn.mean()))
print("The KNeighborsClassifier with k=" + str(knn.n_neighbors) + " has score on test set : \n" + classification_report(y_test,pred_knn))


#####**Confusion Matrices - Πίνακες Σύγχυσης**

H συνάρτηση `plot_cnf_clfs` δέχεται ως ορίσματα τους ταξινομητές μαζί με τα predictions που έγιναν για το test set και με την βοηθεια της matplotlib σχεδιάζει τα confusion matrices για τα εν λόγω αποτελέσματα.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

def plot_cnf_clfs(clfs, preds, y_test, classes):
  for clf, pred in zip(clfs, preds):
    clf_name = str(clf).split("(")[0]
    if (clf_name == "Pipeline"):
        clf = clf.steps[-1][1]

    clf_name = str(clf).split("(")[0]    
    if (clf_name == "DummyClassifier"):
        clf_name = clf_name + " strategy=" + str(clf.strategy)
        if (clf.constant == '+' or clf.constant == '-'):
          clf_name = clf_name + " constant=" + clf.constant
    elif (clf_name == "GaussianNB"):
        clf_name = clf_name
    elif (clf_name == "KNeighborsClassifier"):
        clf_name = clf_name + " n_neighbors=" + (str(clf.n_neighbors))
    
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,pred),display_labels=["+", "-"])
    disp = disp.plot(include_values=True,cmap=plt.cm.Blues, ax=None, xticks_rotation='horizontal')
    disp.ax_.set_title(clf_name)

In [ ]:
#For all classifiers plot the cnf matrix
clfs = [knn, gnb, dc_uniform, dc_constant_minus, dc_constant_plus, dc_most_frequent, dc_stratified]
preds =[pred_knn, pred_gnb, pred_uni, pred_const_minus, pred_const_plus, pred_freq, pred_strat]
plot_cnf_clfs(clfs, preds, y_test,["+", "-"])

#####**Bar Plots για f1 scores**

Η συνάρτηση `barplot_clfs` δέχεται ως ορίσματα τους ταξινομητές μαζί με τα predictions που έγιναν για το test set και με την βοηθεια της matplotlib σχεδιάζει τα bar plot των μετρικών f1_micro avg και f1_macro avg για κάθε ταξινομητη.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def barplot_clfs(clfs, preds, y_test):
  #Bar plot values
  f1_micros = []
  f1_macros = []
  clf_names = []
  #For each classifier find name,f1_micro,f1_macro and save them
  for clf, pred in zip(clfs,preds):
    clf_name = str(clf).split("(")[0]
    if (clf_name == "Pipeline"):
        clf = clf.steps[-1][1]
    clf_name = str(clf).split("(")[0]
    if (clf_name == "DummyClassifier"):
        clf_name = "dc " + clf.strategy
        if (clf.constant == '+' or clf.constant == '-'):
          clf_name = clf_name + clf.constant
    elif (clf_name == "GaussianNB"):
        clf_name = "GNB"
    elif (clf_name == "KNeighborsClassifier"):
       clf_name = "kNN" + " k=" + (str(clf.n_neighbors))
    f1_micro = precision_recall_fscore_support(y_test, pred, average='micro')[2]
    f1_macro = precision_recall_fscore_support(y_test, pred, average='macro')[2]
    f1_micros.append(f1_micro)
    f1_macros.append(f1_macro)
    clf_names.append(clf_name)
  #Create the 1st plot f1_micro
  fig = plt.figure(figsize = (15, 5)) 
  plt.figure(1)
  plt.bar(clf_names, f1_micros,  
        width = 0.4)
  plt.ylabel("f1_micro score") 
  plt.title("f1_micro")
  #Create the 2nd plot f1_macro
  fig = plt.figure(figsize = (15, 5))
  plt.figure(2)
  plt.bar(clf_names, f1_macros,  
        width = 0.4)
  plt.ylabel("f1_macro score") 
  plt.title("f1_macro")
  plt.show()

In [ ]:
barplot_clfs(clfs,preds,y_test)

####**Βελτιστοποίηση ταξινομητών**

Για κάθε ταξινομητή προσπαθούμε να βελτιώσουμε την απόδοση του στο training set με κάποιον συνδιασμό των VarianceThreshold, StandardScaler, MinMaxScaler και PCA.Ως κριτήριο βελτιστοποίησης θα χρησιμοποιήσουμε την μετρική f1_macro όπως είδαμε και στο εργαστήριο.Αφού επιτεχθούν, θα κάνουμε ένα predict στο test set ώστε να δούμε και την απόδοση του βελτιστοποιήμενου ταξινομητή στο test set.

#####**Dummy Classifiers**

Για τη βελτιστοποίηση των dummy ταξινομητών, θα εφαρμόσουμε τα στάδια προεπεξεργασίας VarianceThreshold, StandardScaler και PCA.

Για κάθε ταξινομητή κατασκευάζουμε ένα pipe χρησιμοποιώντας αυτούς τους μετασχηματιστές.

In [ ]:
from sklearn.pipeline import Pipeline
#from imblearn.pipeline import Pipeline

from sklearn.feature_selection import VarianceThreshold
#from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

selector = VarianceThreshold()
#ros = RandomOverSampler()
mscaler = MinMaxScaler()
sscaler = StandardScaler()
pca = PCA()

pipe_uniform = Pipeline(steps=[('selector', selector), ('scaler', sscaler), ('pca', pca)\
                               , ('dc_uniform', dc_uniform)], memory = 'tmp')
pipe_constant_minus = Pipeline(steps=[('selector', selector), ('scaler', sscaler), ('pca', pca)\
                                      , ('dc_constant_minus', dc_constant_minus)], memory = 'tmp')
pipe_constant_plus = Pipeline(steps=[('selector', selector), ('scaler', sscaler), ('pca', pca)\
                                     , ('dc_constant_plus', dc_constant_plus)], memory = 'tmp')
pipe_most_frequent = Pipeline(steps=[('selector', selector), ('scaler', sscaler), ('pca', pca)\
                               , ('dc_most_frequent', dc_most_frequent)], memory = 'tmp')
pipe_stratified = Pipeline(steps=[('selector', selector), ('scaler', sscaler), ('pca', pca)\
                               , ('dc_stratified', dc_stratified)], memory = 'tmp')

Στο στάδιο, αυτό εξετάζουμε τα αποτελέσματα της προεπεξεργασίας με default υπερπαραμέτρους,δηλαδή προτού εφαρμόσουμε grid search/cross-validation, ώστε να επιλέξουμε τις βέλτιστες υπερπαραμέτρους. Ωστόσο όπως παρατηρούμε τα αποτελέσματα παραμένουν ίδια (οι διαφορές στους uniform και stratified ταξινομητές προκύπτουν λόγω της τυχαιότητας αυτών) ανεξάρτητα της επιλογής μετασχηματιστών, οπότε για συντομία εδώ παρουσιάζουμε τα αποτελέσματα μόνο για τον πλήρη συνδυασμό.

In [ ]:
pipes_dc = [pipe_uniform, pipe_constant_minus, pipe_constant_plus, pipe_most_frequent, pipe_stratified]

for pipe in pipes_dc:
  dummy_pipe_name = str(pipe[-1]).split("(")[0]
  dummy_pipe_strategy = pipe[-1].strategy
  dummy_pipe_constant = pipe[-1].constant
  pipe.fit(X_train,y_train)
  pipe_pred = pipe.predict(X_test) 
  if (dummy_pipe_constant == None):
    print("The DummyClassifier with " + dummy_pipe_strategy + " strategy and preprocessing with default parameters has score on test set : \n" + classification_report(y_test,pipe_pred))
  elif (dummy_pipe_constant == "+"):
    print("The DummyClassifier with " + dummy_pipe_strategy + " plus strategy and preprocessing with default parameters has score on test set : \n" + classification_report(y_test,pipe_pred))
  elif (dummy_pipe_constant == "-"):
    print("The DummyClassifier with " + dummy_pipe_strategy + " minus strategy and preprocessing with default parameters has score on test set : \n" + classification_report(y_test,pipe_pred))  
  

Στη συνέχεια θα εκτελέσουμε αναζήτηση πλέγματος στην υπερπαράμετρο threshold του selector και του αριθμού χαρακτηριστικών που δίνει ο PCA. Για να κατασκευάσουμε το κατάλληλο πλέγμα, επισκοπούμε τη διακύμανση των χαρακτηριστικών

In [ ]:
train_variance = X_train.var(axis=0)
train_variance.sort()
print(train_variance)
print(np.max(train_variance))

Στη φάση αυτή, εκτελούμε την αναζήτηση πλέγματος στα pipes των dummy ταξινομητών και εφαρμόζουμε τα αποτελέσματα στα δεδομένα. Δεχόμαστε έτσι τις τελικές μετρικές των βελτιστοποιημένων ταξινομητών.

In [ ]:
import time
from sklearn.model_selection import GridSearchCV

#Grid Search for dummy classifiers
vthreshold = [0, 1e-02, 6e-02, 1e-01, 2e-01, 1]
n_components = [2, 10, 20, 30, 40, 45]

pred_pipes_dc = []
time_pipes_dc = []
for pipe in pipes_dc:
  dummy_name = str(pipe[-1]).split("(")[0]
  dummy_strategy = pipe[-1].strategy
  dummy_constant = pipe[-1].constant
  estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components),\
                                      cv=10, scoring='f1_macro', n_jobs=-1)
  start_time = time.time()
  estimator.fit(X_train, y_train)
  pred = estimator.predict(X_test)
  pred_pipes_dc.append(pred)
  time_pipes_dc.append(time.time() - start_time)
  if (dummy_constant == None):
    print("The optimized DummyClassifier with " + dummy_strategy + " strategy has score on test set : \n" + classification_report(y_test,pred))
  elif (dummy_constant == "+"):
    print("The optimized DummyClassifier with " + dummy_strategy + " plus strategy has score on test set : \n" + classification_report(y_test,pred))
  elif (dummy_constant == "-"):
    print("The optimized DummyClassifier with " + dummy_strategy + " minus strategy has score on test set : \n" + classification_report(y_test,pred))
  print("with best parameters")
  print(estimator.best_params_)
  print("\n")
  after_micro.append(precision_recall_fscore_support(y_test, pred, average='micro')[2])
  after_macro.append(precision_recall_fscore_support(y_test, pred, average='macro', zero_division=0)[2])
  

Παρατηρούμε όμως ότι πάλι τα αποτελέσματα δεν είναι διαφορετικά, παρά την προσπάθεια βελτιστοποίησης. Αυτό όμως είναι και το αναμενόμενο αποτέλεσμα, αφού οι dummy ταξινομητές δεν εξετάζουν τα δεδομένα σε αρκετό βάθος ώστε να έχει σημασία η προεπεξεργασία. Συγκεκριμένα, οι σταθεροί ταξινομητές, προφανώς θα έχουν πάντα την ίδια απόδοση, ενώ και ο most frequent διατηρεί τα ίδια αποτελέσματα, εφόσον η πιο συχνή ετικέτα δεν αλλάζει. Όσο για τους uniform και stratified ταξινομητές, παρατηρείται μία απόκλιση από το αρχικά αποτελέσματα, ωστόσο αυτή είναι κάθε φορά διαφορετική και τυχαία. Εντελώς, στην πρώτη περίπτωση, ενώ στη δεύτερη αφού δεν αφαιρούμε γραμμές από τα δεδομένα μας, οι προβλέψεις θα γίνονται πάντα με την ίδια πιθανότητα.

#####**Gaussian Naives Bayes**

In [ ]:
pipe_gnb = Pipeline(steps=[('selector', selector), ('mscaler', mscaler), ('pca', pca), ('gnb', gnb)],memory="tmp")

Όπως πριν θα εξέτασουμε τα αποτελέσματα της προεπεξεργασίας προτού γίνει αναζήτηση βέλτιστων υπερπαραμέτρων.

In [ ]:
pipe_gnb.fit(X_train, y_train)
pred_pipe_gnb = pipe_gnb.predict(X_test)
score_pipe_gnb = cross_val_score(pipe_gnb, X_train, y_train, cv=10, scoring='f1_macro')

print("The GaussianNB Classifier with default preprocessing has f1_macro score on training set : %f \n" % (score_pipe_gnb.mean()))
print("The GaussianNB Classifier with default preprocessing parameters: \n" + classification_report(y_test,pred_pipe_gnb))

Εδώ βλέπουμε πως ο ίδιος ταξινομητής μαζί με την προεπεξεργασία με τις default υπερπαραμέτρους έχει μικρότερη απόδοση στο training set.Παράλληλα βλέπουμε οτι η απόδοση στο test set έχει επίσης μικρύνει.

Έπειτα θα εφαρμόσουμε αναζήτηση πλέγματος ώστε να βρούμε τις βέλτιστες υπερπαραμέτρους για την προεπεξεργασία για τις οποίες θα έχουμε ακόμα καλύτερη αποδοσή απο τον ταξινομητή χωρίς προεπεξεργασία.

In [ ]:
vthreshold = [0, 1e-02, 6e-02, 1e-01, 2e-01, 1]
n_components = [2, 10, 20, 30, 40, 45]

estimator = GridSearchCV(pipe_gnb, dict(selector__threshold=vthreshold, pca__n_components=n_components), cv=10, scoring='f1_macro',n_jobs=-1)
start_time = time.time()
estimator.fit(X_train,y_train)
pred_pipe_gnb = estimator.predict(X_test)
time_pipe_gnb = time.time()-start_time
print("The optimized GaussianNB Classifier has f1_macro score on training set : %f \n" % (estimator.best_score_))
print("The optimized GaussianNB Classifier : \n" + classification_report(y_test,pred_pipe_gnb))
print("with best parameters")
print(estimator.best_params_)

Θα ελέγξουμε αν υπάρχει περιθώριο βελτίωσης του ταξινομήτη ελέγχοντας κάποιες τιμές κοντά στις τιμές που βρήκαμε, πραγματοποιώντας ουσιαστικά μία progressive αναζήτηση πλέγματος.

In [ ]:
vthreshold = [3e-02, 6e-02, 9e-02]
n_components = [6, 7, 8, 9, 10, 11, 12, 13, 14]

estimator = GridSearchCV(pipe_gnb, dict(selector__threshold=vthreshold, pca__n_components=n_components), cv=10, scoring='f1_macro',n_jobs=-1)
estimator.fit(X_train,y_train)
pred_pipe_gnb = estimator.predict(X_test)
print("The optimized GaussianNB Classifier has f1_macro score on training set : %f \n" % (estimator.best_score_))
print("The optimized GaussianNB Classifier : \n" + classification_report(y_test,pred_pipe_gnb))
print("with best parameters")
print(estimator.best_params_)
after_micro.insert(0, precision_recall_fscore_support(y_test, pred_pipe_gnb, average='micro')[2])
after_macro.insert(0, precision_recall_fscore_support(y_test, pred_pipe_gnb, average='macro', zero_division=0)[2])

Πράγματι βλέπουμε πως έχουμε βελτίωση της απόδοσης του ταξινομητή στο training set σχεδον 2% απο τον απλό GNB.Ενω στο test set λαμβάνουμε απόδοση κοντά στο 90%.

#####**k Nearest Neighbors Classifier (kNN)**

In [ ]:
pipe_knn = Pipeline(steps=[('selector', selector), ('mscaler',mscaler), ('pca', pca), ('knn', knn)],memory="tmp")

Όπως πριν θα εξέτασουμε τα αποτελέσματα της προεπεξεργασίας προτού γίνει αναζήτηση βέλτιστων υπερπαραμέτρων.

In [ ]:
pipe_knn.fit(X_train,y_train)
pred_pipe_knn = pipe_knn.predict(X_test)
score_pipe_knn = cross_val_score(pipe_knn, X_train, y_train, cv=10, scoring='f1_macro')
print("The KNeighborsClassifier with k=" + str(pipe_knn[-1].n_neighbors) + " and default preprocessing parameters has f1_macro score on training set : %f \n" % (score_pipe_knn.mean()))
print("The KNeighborsClassifier with k=" + str(pipe_knn[-1].n_neighbors) + " and default preprocessing parameters has score on test set : \n" + classification_report(y_test,pred_pipe_knn))

Εδώ βλέπουμε πως ο ίδιος ταξινομητής μαζί με την προεπεξεργασία βελτίωσε την απόδοση του σχεδόν κατα 18%.

Έπειτα θα εφαρμόσουμε progressive αναζήτηση πλέγματος ώστε να βελτιστοποιήσουμε τον ταξινομητή επιλέγοντας τις βέλτιστες υπερπαραμέτρους για την προεπεξεργασία και την υπερπαράμετρο k του kNN.

In [ ]:
#Grid for knn classifier
vthreshold = [0, 1e-02, 6e-02, 1e-01, 2e-01, 1]
n_components = [2, 10, 20, 30, 40, 45]
k = [1, 5, 11, 15, 21, 25, 31, 35, 41]

start_time = time.time()
estimator = GridSearchCV(pipe_knn, dict(selector__threshold=vthreshold, pca__n_components=n_components, knn__n_neighbors=k),\
                         cv=10, scoring='f1_macro', n_jobs=-1)
estimator.fit(X_train,y_train)
pred_pipe_knn = estimator.predict(X_test)
time_pipe_knn = time.time() - start_time
print("The optimized KNeighborsClassifier has f1_macro score on training set %f : \n" % (estimator.best_score_))
print("The optimized KNeighborsClassifier has score on test set : \n" + classification_report(y_test,pred_pipe_knn))
print("with best parameters")
print(estimator.best_params_)

Επαναλαμβάνουμε τώρα την αναζήτηση πλέγματος με τιμές υπερπαραμέτρων κοντινές στις βέλτιστες της προηγούμενης αναζήτησης.

In [ ]:
#Grid for knn classifier
vthreshold = [0, 2e-03, 4e-03]
n_components = [17, 18, 19, 20, 21, 22, 23]
k = [3, 5, 7, 9]

estimator = GridSearchCV(pipe_knn, dict(selector__threshold=vthreshold, pca__n_components=n_components, knn__n_neighbors=k),\
                         cv=10, scoring='f1_macro', n_jobs=-1)
estimator.fit(X_train,y_train)
pred_pipe_knn = estimator.predict(X_test)
print("The optimized KNeighborsClassifier has f1_macro score on training set %f : \n" % (estimator.best_score_))
print("The optimized KNeighborsClassifier : \n" + classification_report(y_test,pred_pipe_knn))
print("with best parameters")
print(estimator.best_params_)
after_micro.insert(0, precision_recall_fscore_support(y_test, pred_pipe_knn, average='micro')[2])
after_macro.insert(0, precision_recall_fscore_support(y_test, pred_pipe_knn, average='macro', zero_division=0)[2])

Καταλήγουμε οτι οι βέλτιστες υπερπαράμετροι είναι πολυ κοντά σε αυτές που βρήκαμε και στο προηγούμενο βήμα.Η μόνη υπερπαράμετρος που άλλαξε είναι το vthreshhold ωστόσο η απόδοση έμεινε ίδια.
Οπότε το παρόν βήμα αποτελεί μία επαλήθευση οτι βρισκόμαστε σε καλό συνδυασμό υπερπαραμέτρων.

#####**Confusion Matrices - Πίνακες Σύγχυσης**

In [ ]:
#For all classifiers plot the cnf matrix
clfs = [pipe_knn, pipe_gnb] + (pipes_dc)
preds =[pred_pipe_knn, pred_pipe_gnb] + (pred_pipes_dc) 
plot_cnf_clfs(clfs, preds, y_test,["+", "-"])

#####**Χρόνοι εκτέλεσης**

Ακολουθούν οι χρόνοι που χρειάστηκαν για την εκπαίδευση και εκτίμηση του κάθε ταξινομητή.



In [ ]:
#Create a list with the times for fit and predict
times = [time_pipe_knn] + [time_pipe_gnb] + time_pipes_dc

#Save the names of the classifiers
names = []
for clf in clfs:
  clf_name = str(clf).split("(")[0]
  if (clf_name == "Pipeline"):
      clf = clf.steps[-1][1]
  clf_name = str(clf).split("(")[0]    
  if (clf_name == "DummyClassifier"):
      clf_name = clf_name + " strategy=" + str(clf.strategy)
      if (clf.constant == '+' or clf.constant == '-'):
        clf_name = clf_name + " constant=" + clf.constant
  elif (clf_name == "GaussianNB"):
      clf_name = clf_name
  elif (clf_name == "KNeighborsClassifier"):
      clf_name = clf_name + " n_neighbors=" + (str(clf.n_neighbors))
  names.append(clf_name)

In [ ]:
pd.DataFrame({'Time (s)': times}, index=names)


#####**Bar Plots για f1 scores**

In [ ]:
barplot_clfs(clfs,preds,y_test)

#####**Πίνακας μεταβολής απόδοσης**

Ο παρακάτω πίνακας μας δείχνει την μεταβολή της απόδοσης στο test set (μετρικές f1_micro και f1_macro) πριν και μετά την βελτιστοποίηση των ταξινομητών.



In [ ]:
changes = {
    'Before F1 (micro)': before_micro,
    'After F1 (micro)': after_micro,
    'Before F1 (macro)': before_macro,
    'After F1 (macro)': after_macro
}
pd.DataFrame(changes, index=names)